In [ ]:
import pandas as pd
import numpy as np

In [ ]:
game_df = pd.read_csv("raw_data/Twitch_game_data.csv")
global_df = pd.read_csv("raw_data/Twitch_global_data.csv")
streamers_df = pd.read_csv("raw_data/twitchdata-update.csv")

## Global viewers chart

In [ ]:
Viewers = global_df[["year","Month","Avg_viewers"]]
Viewers["day"] = 1

In [125]:
Viewers = Viewers.set_index(pd.to_datetime(Viewers[["year", "Month", "day"]]))
Viewers = Viewers.drop(["year", "Month", "day"], axis=1)

In [126]:
Viewers.to_csv("data/global_viewers.csv")

## Games views for ridgeplot

In [ ]:
game_df.drop(game_df[game_df["Game"] == "Just Chatting"].index, inplace=True)

In [ ]:
top_games = game_df.groupby(['Game'])['Avg_viewers'].sum().sort_values(ascending=False)[:10].index.to_list()

In [ ]:
games = []

for game in top_games:
    games.append(game_df[game_df["Game"] == game])



In [114]:
games[3]["Game"].iloc[0]

'Counter-Strike: Global Offensive'

In [115]:
for i,game in enumerate(games):
    game.to_csv("data/"+ games[i]["Game"].iloc[0] +".csv")